In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install -U sentence-transformers

In [ ]:
pip install elasticsearch

In [ ]:
import elasticsearch
print(elasticsearch.__version__)

In [ ]:
import time
import sys
import json
import random
import os
from pprint import pprint
from elasticsearch import Elasticsearch, helpers
from sentence_transformers import SentenceTransformer
import numpy as np
from itertools import islice


In [ ]:
class Search:
    def __init__(self):
        PORT="https://5bf9-91-98-61-144.ngrok-free.app"
        AUTH=("elastic", "n0caKGgKTYSNMZtX0sHe")
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.es = Elasticsearch(PORT,basic_auth=AUTH,verify_certs=False)
        client_info = self.es.info()
        print('Connected to Elasticsearch!')
        pprint(client_info.body)

    def get_embedding(self, text):
        return self.model.encode(text)

    def insert_document(self, index, document):
        return self.es.index(index=index, document={
            **document,
            'embedding': self.get_embedding(document['Name']),
        })

    def insert_documents(self, index, documents):
        for document in documents:
            self.insert_document(index,document)

    def insert_LLM_knowledge(self, index, documents):
      for document in documents:
            return self.es.index(index=index, document={
                **document,
                'embedding': self.get_embedding(document['Text']),
            })

    def search(self, index, **query_args):
        return self.es.search(index=index, **query_args)

    def delete_index(self, index):
      return self.es.indices.delete(index=index, ignore=[400, 404])

    def create_embedding_index(self,index):
        self.es.indices.delete(index=index, ignore_unavailable=True)
        self.es.indices.create(index=index, mappings={
            'properties': {
                'embedding': {
                    'type': 'dense_vector',
                }
            }
        })

In [ ]:
es = Search()

In [ ]:
! cp /content/drive/MyDrive/ColabNotebooks/json.json /content/json.json

In [ ]:
with open('json.json', 'rt') as f:
    documents = json.loads(f.read())

In [ ]:
es.create_embedding_index('all_documents')

In [ ]:
es.insert_documents('all_documents',documents)

In [ ]:
query = "KODAK"
es.search('all_documents',
    query={
        'match': {
            'Name': {
                'query': query
            }
        }
    }
)

In [ ]:
query = "KODAK"
es.search('all_documents',
        knn={
            'field': 'embedding',
            'query_vector': es.get_embedding(query),
            'num_candidates': 50,
            'k': 10,
        },

    )

In [ ]:
def Recommender(index,selected_doc):
    results = es.search(index,
        query={
            'multi_match': {
                'query': selected_doc['Name'],
                'fields': ['Name', 'Details'],
            }
        }
    )
    return results['hits']['hits']



In [ ]:
Recommender('all_documents',documents[2])

In [ ]:
selected_docs = random.sample(documents,k = 20)

In [ ]:
es.insert_documents('selected_documents',selected_docs)

In [ ]:
adjacency_dictionary = {}
for document in selected_docs:
    adjacency_dictionary[document["Name"]] = Recommender('selected_documents',document)

In [ ]:
vertices = sorted(adjacency_dictionary.keys())
matrix = [[next((item['_score'] for item in adjacency_dictionary[vertex] if item['_source']['Name'] == v), 0) for v in vertices] for vertex in vertices]
print(matrix)


In [ ]:
query = "Vlogging Camera"
res = es.search('all_documents',
        knn={
            'field': 'embedding',
            'query_vector': es.get_embedding(query),
            'num_candidates': 50,
            'k': 10,
        },

    )
res = res['hits']['hits']
for i in res:
    print(i['_source']['Name'])

In [ ]:
query = "Photography"
res = es.search('all_documents',
        knn={
            'field': 'embedding',
            'query_vector': es.get_embedding(query),
            'num_candidates': 50,
            'k': 10,
        },

    )
res = res['hits']['hits']
for i in res:
    print(i['_source']['Name'])

In [ ]:
query = "video camera"
res = es.search('all_documents',
        knn={
            'field': 'embedding',
            'query_vector': es.get_embedding(query),
            'num_candidates': 50,
            'k': 10,
        },

    )
res = res['hits']['hits']
for i in res:
    print(i['_source']['Name'])

In [ ]:
query = "Kids Camera"
res = es.search('all_documents',
        knn={
            'field': 'embedding',
            'query_vector': es.get_embedding(query),
            'num_candidates': 50,
            'k': 10,
        },

    )
res = res['hits']['hits']
for i in res:
    print(i['_source']['Name'])

In [ ]:
query = "Camera 4K"
res = es.search('all_documents',
        knn={
            'field': 'embedding',
            'query_vector': es.get_embedding(query),
            'num_candidates': 50,
            'k': 10,
        },

    )
res = res['hits']['hits']
for i in res:
    print(i['_source']['Name'])

In [ ]:
testing_documents = documents[:5]

In [ ]:
es.insert_documents('testing_documents',testing_documents)

In [ ]:

adjacency_dictionary = {}
for document in testing_documents:
    adjacency_dictionary[document["Name"]] = Recommender('testing_documents',document)

In [ ]:
vertices = sorted(adjacency_dictionary.keys())
prediction_matrix = [[next((item['_score'] for item in adjacency_dictionary[vertex] if item['_source']['Name'] == v), 0) for v in vertices] for vertex in vertices]
print(prediction_matrix)

In [ ]:
ground_truth = [[1,4,3,5,2],
                [2,1,5,4,3],
                [3,4,1,2,5],
                [3,5,2,1,4],
                [2,3,5,4,1]]

In [ ]:
def rank_predictions(prediction_matrix):
    ranked_predictions = []
    for i, row in enumerate(prediction_matrix):
        ranked_predictions.append(row)
        ranked_predictions[i] = sorted(ranked_predictions[i], reverse = True)
        for j in range (len(prediction_matrix)):
            indx = ranked_predictions[i].index(row[j])
            row [j] = indx + 1
    return  prediction_matrix

def apk(actual, predicted, k):
    if not actual:
        return 0.0

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p == actual[i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k):
    predicted = rank_predictions(predicted)
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
mapk(ground_truth,  prediction_matrix, 5)

In [ ]:
def DCG_at_k(rel_vector, k):
    rel_vector = np.asarray(rel_vector)[:k]
    discounts = np.log2(np.arange(len(rel_vector)) + 2)
    print("Discounts:", discounts)
    print("Rel Vector:", rel_vector)
    return np.sum((np.power(2, rel_vector) - 1) / discounts)

def NDCG_at_k(ground_truth, predictions, k):
    ground_truth = np.array(ground_truth)
    predictions = np.array(predictions)
    rel_scores_ground_truth = ground_truth.flatten()
    rel_scores_predictions = predictions.flatten()

    DCG_ground_truth = DCG_at_k(rel_scores_ground_truth, k)
    DCG_predictions = DCG_at_k(rel_scores_predictions, k)

    ideal_ground_truth = np.sort(rel_scores_ground_truth)[::-1]
    IDCG_ground_truth = DCG_at_k(ideal_ground_truth, k)
    print("DCG Ground Truth:", DCG_ground_truth)
    print("DCG Predictions:", DCG_predictions)
    print("IDCG Ground Truth:", IDCG_ground_truth)

    if IDCG_ground_truth == 0:
        return 0.
    return DCG_predictions / IDCG_ground_truth

In [ ]:
k = 5
ndcg = NDCG_at_k(ground_truth, rank_predictions(prediction_matrix), k)
print(ndcg)

In [ ]:
from huggingface_hub import interpreter_login

interpreter_login()

In [ ]:
!git clone https://github.com/google/sentencepiece.git
!cd sentencepiece
!mkdir build
!cd build
!cmake ..
!make -j $(nproc)
!sudo make install
!sudo ldconfig -v

In [ ]:
import torch
# print(torch.cuda.current_device())
print(torch.cuda.is_available())
print("Torch version:",torch.__version__)

In [ ]:
!pip install optimum
!pip install auto-gptq
!pip install transformers

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install accelerate

In [ ]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4


In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [ ]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=2048, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32,
    n_ctx=2048

    )


In [ ]:
lcpp_llm.params.n_gpu_layers

In [ ]:
prompt = "تعداد اعضای هیات علمی دانشگاه؟"
system_prompt = "دانشگاه فردوسی مشهد از دانشگاه‌های ملی[۱] و زیرمجموعه وزارت علوم، تحقیقات و فناوری ایران است که بر اساس نام حکیم ابوالقاسم فردوسی، شاعر و حماسه‌سرای ایرانی نام‌گذاری شده‌است. این دانشگاه که پیشنهاد بنیانگذاری آن به پیش از دهه بیست خورشیدی برمی‌گردد، سومین دانشگاه بنیان گذاشته شده در ایران به‌شمار می‌آید که در سال ۱۳۹۸ جشن هفتادسالگی خود را برگزار کرد.[۲] دانشگاه فردوسی مشهد در زمان پهلوی از نظر علمی و آکادمیک زیر نظر دانشگاه جرج‌تاون قرار داشت.[۳] دانشگاه فردوسی مشهد با دارا بودن ۱۳ دانشکده، ۸ پژوهشکده،[۴] بیش از ۲۵۰۰۰ دانشجوی داخلی و بین‌المللی و ۸۱۴ عضو هیئت علمی،[۵] به عنوان یکی از بزرگ‌ترین دانشگاه‌های ایران و قطب علمی شرق کشور محسوب می‌گردد.[۶]"


prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. You know that {system_prompt}.

USER:  {prompt}

ASSISTANT:
'''

In [ ]:
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

In [ ]:
print(response["choices"][0]["text"])

In [ ]:
query = "canon"
res = es.search('all_documents',
knn={
  'field': 'embedding',
  'query_vector': es.get_embedding(query),
  'num_candidates': 50,
  'k': 10,
},

)
res = res['hits']['hits']
print(res)

In [ ]:
def askLLM_context(prompt):
    context = ""
    query = f"{prompt}"
    res = es.search('all_documents',
            knn={
                'field': 'embedding',
                'query_vector': es.get_embedding(query),
                'num_candidates': 50,
                'k': 3,
            },

        )['hits']['hits']
    print(len(res))
    for i in res:
        context =context +"Name:" +i['_source']['Name']+"+"
        context =context +"Price:"+i['_source']['Price']+ "+"
        context =context +"Details:"+"".join(i['_source']['Details'][0])
    print(len(context))

    res = [context[idx:idx + 1000] for idx in range(len(context) - 1000 + 1)]
    print(len(res))
    LLM_documents = []
    for i in res:
      LLM_documents.append({'Name':i})
    print(len(LLM_documents))
    res = es.insert_documents('llm',LLM_documents)
    res = es.search('llm',
            knn={
                'field': 'embedding',
                'query_vector': es.get_embedding(query),
                'num_candidates': 50,
                'k': 2,
            },

        )['hits']['hits']
    LLM_context = ""

    for i in res:
        LLM_context =LLM_context + i['_source']['Name']+ "  "
    prompt_template = f'''SYSTEM: You are a helpful, respectful and honest assistant. You know that {LLM_context}.

    USER:  {prompt}

    ASSISTANT:
    '''
    response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)
    print(response["choices"][0]["text"])
    es.delete_documents('llm')


In [ ]:
ask_LLM("i want to buy a canon camera with price under 100 dollars")

In [ ]:
ask_LLM("i want to buy a digital camera")

In [ ]:
ask_LLM("i want to buy a 4k camera")

In [ ]:
ask_LLM("i want to buy a waterproof camera")

In [ ]:
ask_LLM("i want to buy a vlogging camera")